# Problema 1

O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes $$a, b > 0$$ e devolve inteiros $$r, s, t$$ tais que $$a \cdot s + b \cdot t = r$$ e $$r = \gcd(a, b)$$.

Para além das variáveis $$r, s, t$$, o código requer 3 variáveis adicionais $$r', s', t'$$ que representam os valores de $$r, s, t$$ no “próximo estado”.

**Algoritmo:**

```plaintext
INPUT a, b
assume a > 0 and b > 0
r, r', s, s', t, t' = a, b, 1, 0, 0, 1
while r' != 0
  q = r div r'
  r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t'
OUTPUT r, s, t


### a) Construa um SFOTS usando BitVector’s de tamanho $$n$$ que descreva o comportamento deste programa.  Considere estado de erro quando $$\,r=0\,$$ ou alguma das variáveis atinge o “overflow”.

# Objetivo


O objétivo do modelo:
1. **Cálculo do Máximo Divisor Comum (MDC)** entre dois inteiros `a` e `b` utilizando o algoritmo estendido de Euclides.
2. Acompanhar os estados intermédios (`r`, `r_linha`, `s`, `s_linha`, `t`, `t_linha`) e as transições associadas.
3. Identificação das condições de erro:
    - Quando o valor de `r` é igual a zero;
    - Quado ocorre **overflow** nos registos.

In [8]:
from pysmt.shortcuts import *
from pysmt.typing import BVType, INT

def gera_estado(vars, s, i, n):
    estado = {var: Symbol(f"{var}!{s}{i}", BVType(n)) for var in vars}
    estado['pc'] = Symbol(f"pc!{s}{i}", INT)
    return estado 

def estado_inicial(estado, a, b, n):
    return And(
        Equals(estado['r'], a),
        Equals(estado['r_linha'], b),
        Equals(estado['s'], BV(1, n)),
        Equals(estado['s_linha'], BV(0, n)),
        Equals(estado['t'], BV(0, n)),
        Equals(estado['t_linha'], BV(1, n)),
        Equals(estado['q'], BV(0, n)),
        BVUGT(a, BV(0, n)),
        BVUGT(b, BV(0, n)),
        Equals(estado['pc'], Int(0))
    )

def erro(estado):
    return Equals(estado['pc'], Int(3))

def transicao(atual, prox):
    n = atual['r'].symbol_type().width

    def transP():
        return And(
            Or(Equals(atual['pc'], Int(0)), Equals(atual['pc'], Int(1))),
            Not(Equals(atual['r_linha'], BV(0, n))),
            Equals(prox['q'], BVUDiv(atual['r'], atual['r_linha'])),
            Equals(prox['pc'], Int(1) - atual['pc']),
            Equals(prox['r'], atual['r_linha']),
            Equals(prox['r_linha'], BVSub(atual['r'], BVMul(atual['q'], atual['r_linha']))),
            Equals(prox['s'], atual['s_linha']),
            Equals(prox['s_linha'], BVSub(atual['s'], BVMul(atual['q'], atual['s_linha']))),
            Equals(prox['t'], atual['t_linha']),
            Equals(prox['t_linha'], BVSub(atual['t'], BVMul(atual['q'], atual['t_linha'])))
        )

    def erro0():
        return And(
            Equals(atual['pc'], Int(1)),
            Equals(atual['r'], BV(0, n)),
            Equals(prox['pc'], Int(3))
        )

    def overflow():
        overflow_checks = [
            BVUGT(atual[var], BV(2**n - 1, n))
            for var in ['r', 'r_linha', 's', 's_linha', 't', 't_linha', 'q']
        ]
        return And(
            Equals(atual['pc'], Int(1)),
            Or(*overflow_checks),
            Equals(prox['pc'], Int(3))
        )

    def estado_final():
        return And(
            Equals(atual['pc'], Int(1)),
            Equals(atual['r_linha'], BV(0, n)),
            Equals(prox['pc'], Int(2)),
            *(Equals(prox[var], atual[var]) for var in ['r', 'r_linha', 's', 's_linha', 't', 't_linha', 'q'])
        )
 
    return Or(
        transP(),
        erro0(),
        overflow(),
        estado_final()
    )

def gera_trace(vars, estado_inicial, transicao, erro, n, N):
    with Solver(name="z3") as solver:
        S = [gera_estado(vars, 'S', i, n) for i in range(N+1)]
        a = Symbol('a_input', BVType(n))  # Nome único para evitar conflitos
        b = Symbol('b_input', BVType(n))  # Nome único para evitar conflitos
        I = estado_inicial(S[0], a, b, n)
        traces = [transicao(S[i], S[i+1]) for i in range(N)]

        estado_final = Equals(S[N]['pc'], Int(2))

        if solver.solve([I, And(traces), estado_final]):
            model = solver.get_model()
            for i in range(N+1):
                print(f"Estado S{i}:")
                for v in S[i]:
                    if S[i][v] in model:
                        valor = model[S[i][v]].constant_value()
                        print(f"        {v} = {valor}")
        else:
            print("Nenhum Modelo Encontrado!")

vars = ['r', 'r_linha', 's', 's_linha', 't', 't_linha', 'q']
gera_trace(vars, estado_inicial, transicao, erro, 8, 8)

Estado S0:
        r = 96
        r_linha = 216
        s = 1
        s_linha = 0
        t = 0
        t_linha = 1
        q = 0
        pc = 0
Estado S1:
        r = 216
        r_linha = 96
        s = 0
        s_linha = 1
        t = 1
        t_linha = 0
        q = 0
        pc = 1
Estado S2:
        r = 96
        r_linha = 216
        s = 1
        s_linha = 0
        t = 0
        t_linha = 1
        q = 2
        pc = 0
Estado S3:
        r = 216
        r_linha = 176
        s = 0
        s_linha = 1
        t = 1
        t_linha = 254
        q = 0
        pc = 1
Estado S4:
        r = 176
        r_linha = 216
        s = 1
        s_linha = 0
        t = 254
        t_linha = 1
        q = 1
        pc = 0
Estado S5:
        r = 216
        r_linha = 216
        s = 0
        s_linha = 1
        t = 1
        t_linha = 253
        q = 0
        pc = 1
Estado S6:
        r = 216
        r_linha = 216
        s = 1
        s_linha = 0
        t = 253
        t_linha = 1
  

### 1) Estrutura:

### 1.1) Componentes:

- Variáveis de Estado:
    
    - `r` e `r_linha`: Variáveis de Estado para o cálculo do MDC;
    
    - `s`, `s_linha`, `t`, `t_linha`: Coeficientes de Bèzout;  

    - `q`: Quoeficiente da Divisão;

    - `pc`: "Program Counter"

### 1.2) Funções Principais:

- `def gera_estados(vars, s, i, n)`:

Esta função gera um novo estado do sistema com variáveis simbólicas, sendo os seus parametros com uma lista de variáveis, prefixo do estado, indice e o número de bits.

- `def estado_inicial(estado, a, b, n)`:

Esta função define as condições iniciais do sistema, inicializando todas as variáveis com os seus valores iniciais. 

Estabelece a Pré-condição:

### (a > 0 && b > 0)

- `def transicao(atual, prox)`

A função representada define as regras para a transição de estados, implementa a lógica principal do algoritmo e inclui verificações de erro caso `r' == 0` e em caso de _overflow_

- 


      

### b) Prove, usando a metodologia dos invariantes interpolantes, que o modelo nunca atinge o estado de erro

In [ ]:
from pysmt.shortcuts import *
from pysmt.typing import BVType

def extract_base_name(symbol):
    """Extrai o nome base de uma variável."""
    return symbol.split('!')[0]

def rename_expression(expr, estado):
    """Renomeia variáveis em uma expressão com base no estado."""
    substitutions = {v: estado[extract_base_name(v.symbol_name())] for v in get_free_variables(expr)}
    return expr.substitute(substitutions)

def compare_states(state1, state2):
    """Compara dois estados."""
    return And([Equals(state1[var], state2[var]) for var in state1])

def reverse_transition(transition_func):
    """Inverte uma função de transição."""
    return lambda curr, next: transition_func(next, curr)

def create_state(vars, prefix, index, bit_width):
    """Cria um estado com variáveis nomeadas."""
    return {v: Symbol(f"{v}!{prefix}{index}", BVType(bit_width)) for v in vars}

def initial_conditions(estado, a, b, bit_width):
    """Define as condições iniciais."""
    return And(
        Equals(estado['r'], a),
        Equals(estado['r_linha'], b),
        Equals(estado['s'], BV(1, bit_width)),
        Equals(estado['s_linha'], BV(0, bit_width)),
        Equals(estado['t'], BV(0, bit_width)),
        Equals(estado['t_linha'], BV(1, bit_width)),
        BVUGT(a, BV(0, bit_width)),
        BVUGT(b, BV(0, bit_width)),
        Equals(estado['pc2'], BV(0, bit_width))
    )

def error_condition(estado, bit_width):
    """Define a condição de erro."""
    return Equals(estado['pc2'], BV(3, bit_width))

def transition_function(curr, next):
    """Define a função de transição entre estados."""
    bit_width = curr['r'].symbol_type().width
    q = BVUDiv(curr['r'], curr['r_linha'])

    return Or(
        # Transições principais
        And(
            Or(Equals(curr['pc2'], BV(0, bit_width)), Equals(curr['pc2'], BV(1, bit_width))),
            Not(Equals(curr['r_linha'], BV(0, bit_width))),
            Equals(next['pc2'], BV(1, bit_width) - curr['pc2']),
            Equals(next['r'], curr['r_linha']),
            Equals(next['r_linha'], BVSub(curr['r'], BVMul(q, curr['r_linha']))),
            Equals(next['s'], curr['s_linha']),
            Equals(next['s_linha'], BVSub(curr['s'], BVMul(q, curr['s_linha']))),
            Equals(next['t'], curr['t_linha']),
            Equals(next['t_linha'], BVSub(curr['t'], BVMul(q, curr['t_linha']))),
            Equals(curr['q'], q)
        ),
        # Condição de erro
        And(
            Equals(curr['pc2'], BV(1, bit_width)),
            Equals(curr['r'], BV(0, bit_width)),
            Equals(next['pc2'], BV(3, bit_width))
        ),
        # Overflow
        And(
            Equals(curr['pc2'], BV(1, bit_width)),
            Or(*(BVUGT(curr[var], BV(2**bit_width - 1, bit_width)) for var in curr)),
            Equals(next['pc2'], BV(3, bit_width))
        ),
        # Estado final
        And(
            Equals(curr['pc2'], BV(1, bit_width)),
            Equals(curr['r_linha'], BV(0, bit_width)),
            Equals(next['pc2'], BV(2, bit_width)),
            *(Equals(next[var], curr[var]) for var in curr)
        )
    )

def run_model_checking(vars, init_func, trans_func, error_func, bit_width, max_states, max_steps):
    """Executa a verificação de modelos no sistema."""
    with Solver(name="z3") as solver:
        X_states = [create_state(vars, "X", i, bit_width) for i in range(max_states + 1)]
        Y_states = [create_state(vars, "Y", i, bit_width) for i in range(max_steps + 1)]
        reversed_trans = reverse_transition(trans_func)

        for n in range(1, max_states + 1):
            for m in range(1, max_steps + 1):
                # Inicialização e transições
                initial_state = init_func(X_states[0], Symbol('a', BVType(bit_width)), Symbol('b', BVType(bit_width)), bit_width)
                transitions = And([trans_func(X_states[i], X_states[i + 1]) for i in range(n)])
                Rn = And(initial_state, transitions)

                # Erros e estados reversos
                error_state = error_func(Y_states[0], bit_width)
                reverse_transitions = And([reversed_trans(Y_states[i], Y_states[i + 1]) for i in range(m)])
                Um = And(error_state, reverse_transitions)

                # Verificação de segurança
                unsafe_condition = And(Rn, compare_states(X_states[n], Y_states[m]), Um)
                if solver.solve([unsafe_condition]):
                    print("> Sistema inseguro encontrado")
                    return
                
                print("> Nenhum erro encontrado para n =", n, "e m =", m)

    print("> Verificação concluída: O sistema é seguro")

# Variáveis e execução
vars = ['r', 'r_linha', 's', 's_linha', 't', 't_linha', 'pc2', 'q']
run_model_checking(vars, initial_conditions, transition_function, error_condition, 8, 8, 8)


> Nenhum erro encontrado para n = 1 e m = 1
> Nenhum erro encontrado para n = 1 e m = 2
> Nenhum erro encontrado para n = 1 e m = 3
> Nenhum erro encontrado para n = 1 e m = 4
> Nenhum erro encontrado para n = 1 e m = 5
> Nenhum erro encontrado para n = 1 e m = 6
> Nenhum erro encontrado para n = 1 e m = 7
> Nenhum erro encontrado para n = 1 e m = 8
> Nenhum erro encontrado para n = 2 e m = 1
> Nenhum erro encontrado para n = 2 e m = 2
> Nenhum erro encontrado para n = 2 e m = 3
> Nenhum erro encontrado para n = 2 e m = 4
> Nenhum erro encontrado para n = 2 e m = 5
> Nenhum erro encontrado para n = 2 e m = 6
> Nenhum erro encontrado para n = 2 e m = 7
> Nenhum erro encontrado para n = 2 e m = 8
> Nenhum erro encontrado para n = 3 e m = 1
> Nenhum erro encontrado para n = 3 e m = 2
> Nenhum erro encontrado para n = 3 e m = 3
> Nenhum erro encontrado para n = 3 e m = 4
> Nenhum erro encontrado para n = 3 e m = 5
> Nenhum erro encontrado para n = 3 e m = 6
> Nenhum erro encontrado para n 